*Sistema de Recomendação Netflix - DSA*

In [ ]:
# Bibliotecas necessárias

import os
import random
import numpy as np 
import pandas as pd
import seaborn as sns 
import matplotlib 
import matplotlib.pyplot as plt
import scipy 
import sklearn 
from scipy import sparse 
from scipy.sparse import csr_matrix 
from sklearn.decomposition import TruncatedSVD 
from sklearn.metrics.pairwise import cosine_similarity 
from datetime import datetime 

In [ ]:
#Formatação dos gráficos

matplotlib.use('nbAgg')
plt.rcParams.update({'figure.max_open_warning':0})
sns.set_style('whitegrid')

Carregando os dados

In [ ]:
 
#Marca o inicio da execução de leitura dos arquivos

star = datetime.now()

In [ ]:
#Cria um arquivo final chamado dados_netflix.csv

#Se o arquivo não existir, cria-se o arquivo em modo de escrita(w)

if not os.path.isfile('C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/dados_netflix.csv'):
    #cria e abre o arquivo para gravação
    dataset = open('C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/dados_netflix.csv',mode = 'w')
    #Lista para as linhas de arquivo 
    linhas = list()
    #nomes e caminhos dos arquivos
    arquivos = ['C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/combined_data_1.txt',
                'C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/combined_data_2.txt',
                'C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/combined_data_3.txt',
                'C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/combined_data_4.txt']
    #loop por cada arquivo na lista de arquivos
    for arquivo in arquivos:
        print("Lendo arquivo{}...".format(arquivo))
        #com o arquivo aberto extraimos as linhas
        with open(arquivo) as f:
            for linha in f:
                del linhas[:]
                #divide a linha do arquivo pelo caracter de final de linha
                linha = linha.strip()
                #se encontrarmos ":" fazemos o replace removendo o caracter, pois queremos apenas o ID do filme
                if linha.endswith(':'):
                    movie_id = linha.replace(':','')
                #se não, criamos um list comprehension para separar as colunas por virgula
                else:
                    linhas = [x for x in linha.split(',')]
                    linhas.insert(0, movie_id)
                    dataset.write(','.join(linhas))
                    dataset.write('\n')
        print('Concluído.\n')
    dataset.close()
        
    

In [ ]:
#Imprime o tempo de processamento
print('Tempo total de processamento: ', datetime.now()-star)

In [ ]:
#Carregando o Dataframe

df_netflix = pd.read_csv('C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/dados_netflix.csv',sep=',', names = ['movie','user','rating','date'])
df_netflix.date = pd.to_datetime(df_netflix.date)
print('Conluído!')

In [ ]:
#Ordenando o DF por data
df_netflix.sort_values(by='date', inplace = True)
print("Concluído!")

In [ ]:
#Shape dos dados
df_netflix.shape

In [ ]:
#Visualizando os primeiros dados

df_netflix.head()

*ANÁLISE EXPLORATÓRIA DE DADOS*

In [ ]:
#Resumo dos dados

print("Resumo dos Dados")
print('-'*50)
print('Número total de filmes: ', len(np.unique(df_netflix.movie)))
print('Número total de usuários: ', len(np.unique(df_netflix.user)))
print('Número total de avaliações: ', df_netflix.shape[0])


In [ ]:
total_users = len(np.unique(df_netflix.user))
total_movies = len(np.unique(df_netflix.movie))

In [ ]:
#Verificando as estatisticas da coluna de avaliação (Rating)
df_netflix.describe()['rating']

In [ ]:
#Verificando valores ausentes
sum(df_netflix.isnull().any())

In [ ]:
#Verificando se há valores duplicados (não consideramos a data)
sum(df_netflix.duplicated(['movie','user','rating']))

Antes de seguir com a EDA, faremos o split dos dados

In [ ]:
#Criaremos um dataset em disco com os dados de treino, para que não haja necessidade de executar todo
#o processo de carga novamente a cada vez que executar o notebook

if not os.path.isfile('C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/dados_netflix_treino.csv'):
    df_netflix.iloc[:int(df_netflix.shape[0]*0.80)].to_csv('C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/dados_netflix_treino.csv', index = False)


In [ ]:
#Criaremos um dataset em disco com os dados de teste, para que não haja necessidade de executar todo
#o processo de carga novamente a cada vez que executar o notebook

if not os.path.isfile('C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/dados_netflix_teste.csv'):
    df_netflix.iloc[int(df_netflix.shape[0]*0.80):].to_csv('C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/dados_netflix_teste.csv', index = False)
    

In [ ]:
#Para liberar memoria, deletaremos o dataset original
del df_netflix

In [ ]:
#Carregando os dados de treino e teste em dataframe do pandas
df_netflix_treino = pd.read_csv('C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/dados_netflix_treino.csv', parse_dates = ['date'])
df_netflix_teste = pd.read_csv('C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/dados_netflix_teste.csv')

In [ ]:
#Resumo dos dados de treino

print("Resumo dos Dados Treino")
print('-'*50)
print('Número total de filmes: ', len(np.unique(df_netflix_treino.movie)))
print('Número total de usuários: ', len(np.unique(df_netflix_treino.user)))
print('Número total de avaliações: ', df_netflix_treino.shape[0])

In [ ]:
#Função para ajustar as unidades de medida

def ajusta_unidades(num, units = 'M'):
    units = units.lower()
    num = float(num)
    if units == 'k':
        return str(num/10**3)+"K"
    if  units == 'm':
        return str(num/10**6)+"M"
    if  units == 'b':
        return str(num/10**9)+"B"

In [ ]:
#Supressão de warnings
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')

In [ ]:
#Verificando a distribuição dos dados de 'Rating'
fig, ax = plt.subplots()
sns.countplot(df_netflix_treino.rating)
plt.title('Distribuição das Avaliações nos Dados de Treino', fontsize = 15)
ax.set_yticklabels([ajusta_unidades(item, 'M') for item in ax.get_yticks()])
ax.set_ylabel('Número de Avaliações (em Milhões)')
plt.show()

Vamos verificar se o dia da semana tem influência na avaliação do usuário.
Para isso precisaremos incluir uma nova coluna com o dia da semana


In [ ]:
#Extrai o dia da semana e grava em nova coluna
df_netflix_treino['dia_semana'] = df_netflix_treino['date'].dt.strftime('%A')
df_netflix_treino.head()

In [ ]:
#Plot
fig, ax = plt.subplots()
sns.countplot(x = 'dia_semana', data = df_netflix_treino, ax = ax)
plt.title('Número de Avaliações por Dia da Semana', fontsize = 15)
plt.ylabel('Total de Avaliações')
plt.xlabel('')
ax.set_yticklabels([ajusta_unidades(item,'M') for item in ax.get_yticks()])
plt.show()

In [ ]:
#Média das avaliações por dia da semana
media_dia_semana = df_netflix_treino.groupby(by = ['dia_semana'])['rating'].mean()
print('Média de avaliações')
print('-'*30)
print(media_dia_semana)


o dia da semana parece não ter influencia na avaliação dos usuários

In [ ]:
#Análise das avaliações ao longo do tempo
fig = plt.figure(figsize=plt.figaspect(.45))
ax = df_netflix_treino.resample('m',on = 'date')['rating'].count().plot()
ax.set_title('Número de Avaliações por mês')
plt.xlabel('Mês')
plt.ylabel('Número de Avaliações por Mês')
ax.set_yticklabels([ajusta_unidades(item,'M') for item in ax.get_yticks()])
plt.show()

In [ ]:
#Usuários que mais avaliaram
num_aval_user = df_netflix_treino.groupby(by='user')['rating'].count().sort_values(ascending=False)
num_aval_user.head()

In [ ]:
#Resumo estatístico
num_aval_user.describe()

In [ ]:
#Função de densidade de probabilidade  
fig =  plt.figure(figsize= plt.figaspect(.45))
ax1 = plt.subplot(121)
sns.kdeplot(num_aval_user, shade = True, ax=ax1)
plt.xlabel('Número de Avaliações por Usuários')
plt.title('PDF - Função Densidade de Probabilidade')
ax2 = plt.subplot(122)
sns.kdeplot(num_aval_user, shade = True, cumulative = True, ax = ax2)
plt.xlabel('Número de Avaliações por Usuário')
plt.title('CDF - Função de Densidade Acumulada')
plt.show()


In [ ]:
#Extraindo os percentis
percentis = num_aval_user.quantile(np.arange(0,1.01,0.01), interpolation='higher')

In [ ]:
percentis[::5]

In [ ]:
#Plot
fig = plt.figure(figsize= plt.figaspect(.45))
plt.title("Percentis")
percentis.plot()

#Quartis com diferença de 0.05
plt.scatter(x = percentis.index[::5],
            y = percentis.values[::5],
            c = 'orange',
            label = 'Percentis com intervalo de 0.05')

#Quartis com diferença de 0.25
plt.scatter(x = percentis.index[::25],
            y = percentis.values[::25],
            c = 'm',
            label = 'Percentis com intervalo de 0.25')

#Label e legenda
plt.ylabel('Número de Avaliações por Usuário')
plt.xlabel('Valor de Percentis')
plt.legend(loc = 'best')

#Marca os percentis 25, 50, 75 e 100
for x, y in zip(percentis.index[::25],percentis[::25]):
    plt.annotate(s = '({} , {})'.format(x,y) , xy = (x, y), xytext = (x - 0.05, y+500), fontweigth = 'bold')
plt.show()

Criação de Matriz Esparsa Treino



In [ ]:
#Cria a matriz em formato de numpy caso não exista
#Se existe apenas carregamos
if os.path.isfile('C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/matriz_esparsa_treino.npz'):
    matriz_esparsa_treino = sparse.load_npz('C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/matriz_esparsa_treino.npz')
    print('Matriz Carregada!')
else:
    matriz_esparsa_treino = sparse.csr_matrix((df_netflix_treino.rating.values, (df_netflix_treino.user.values,
                                                                                 df_netflix_treino.movie.values)),)
print('Matriz Criada, o shape é: (user, movie):', matriz_esparsa_treino.shape)
sparse.save_npz('C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/matriz_esparsa_treino.npz', matriz_esparsa_treino)
print('Matriz salva em disco')


In [ ]:
#Calculando a esparsidade da matriz
linhas, colunas = matriz_esparsa_treino.shape
elementos_nonzero = matriz_esparsa_treino.count_nonzero()
print("Esparsidade da matriz de treino: {} %".format(( 1 - (elementos_nonzero/ (linhas * colunas))) * 100))

Criando a matriz esparsa de teste


In [ ]:
#Cria a matriz em formato de numpy caso não exista
#Se existe apenas carregamos
if os.path.isfile('C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/matriz_esparsa_teste.npz'):
    matriz_esparsa_teste = sparse.load_npz('C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/matriz_esparsa_teste.npz')
    print('Matriz Carregada!')
else:
    matriz_esparsa_teste = sparse.csr_matrix((df_netflix_teste.rating.values, (df_netflix_teste.user.values,
                                                                                 df_netflix_teste.movie.values)),)
print('Matriz Criada, o shape é: (user, movie):', matriz_esparsa_teste.shape)
sparse.save_npz('C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/matriz_esparsa_teste.npz', matriz_esparsa_teste)
print('Matriz salva em disco')

In [ ]:
#Calculando a esparsidade da matriz
linhas, colunas = matriz_esparsa_teste.shape
elementos_nonzero = matriz_esparsa_teste.count_nonzero()
print("Esparsidade da matriz de teste: {} %".format(( 1 - (elementos_nonzero/ (linhas * colunas))) * 100))

Calculando a média global de todas as avaliações de filme, avaliação por usuário, avaliação média por filme

In [ ]:
#Média global de todas as avaliações de usuários
medias = dict()
media_global = matriz_esparsa_treino.sum() / matriz_esparsa_treino.count_nonzero()
medias['global'] = media_global
medias 

Construção da função para cálculo da média de avaliações

In [ ]:
#Função de cálculo da média
def calc_media_aval(sparse_matrix, of_users):
    #1 eixo do usuario
    #0 eixo do filme
    ax = 1 if of_users else 0
    
    #soma
    sum_of_ratings = sparse_matrix.sum(axis=ax).A1
    
    #Matriz boolena de avaliações
    is_rated = sparse_matrix !=0
    
    #Numero de avaliações de cada usuário ou filme
    no_of_ratings = is_rated.sum(axis=ax).A1
    
    #Ids maximos de usuarios e filmes na matriz esparsa
    u, m = sparse_matrix.shape
    
    #dicionario de usuarios e suas avaliaçoes medias
    media_aval = {i: sum_of_ratings[i]/no_of_ratings[i] for i in range(u if of_users else m) if no_of_ratings[i] !=0}
    
    #Retorna o dict media_aval
    return media_aval

In [ ]:
#Medias de avaliação por usuário
medias['user'] = calc_media_aval(matriz_esparsa_treino, of_users=True)

In [ ]:
#Medias de avaliação por filme
medias['movie'] = calc_media_aval(matriz_esparsa_treino, of_users=False)

In [ ]:
#Podemos verificar um usuário qualquer
medias['user'][149]

In [ ]:
#Podemos verificar um filme qualquer
medias['movie'][32]

PDFs e CDs da média. Avaliações de usuários e filmes (dados treino)

In [ ]:
#plot
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize= plt.figaspect(.45))
fig.suptitle('Média de avaliações por usuários e por filme:', fontsize=15)

ax1.set_title('Média de Avaliações de Usuários')

#Obternis a lista de avaliações médias do usuário no dicionário de médias
medias_usuarios = [rat for rat in medias['user'].values()]
sns.distplot(medias_usuarios, ax= ax1, hist = False, kde_kws=dict(cumulative = True), label='CDF')
sns.distplot(medias_usuarios, ax= ax1, hist = False, label='PDF')

#obtemos a lista de avaliações médias de filme no dicionário
medias_filme = [rat for rat in medias['movie'].values()]
sns.distplot(medias_usuarios, ax= ax2, hist = False, kde_kws=dict(cumulative = True), label='CDF')
sns.distplot(medias_usuarios, ax= ax2, hist = False, label='PDF')

plt.show()



*PROBLEMA DO COLD START*

In [ ]:
#Cold start de usuários
usuarios_treino = len(medias['user'])
novos_usuarios = total_users - usuarios_treino

In [ ]:
#Print 
print('Total Geral de Usuários:', total_users)
print('Total Geral de Usuários em Treino:', usuarios_treino)
print('Total Geral de Usuários que NÃO estão em treino: {} ({}%)'.format(novos_usuarios,
                                                                         np.round((novos_usuarios / total_users)*100, 2)))

75148 usuários nao fazem parte dos dados de treino, ou seja, não temos como aprender o padrão de avaliação desses usuário. esse é o problema do cold start.

In [ ]:
#Cold start de usuários
filmes_treino = len(medias['movie'])
novos_filmes = total_movies - filmes_treino

In [ ]:
#Print 
print('Total Geral de Filmes:', total_movies)
print('Total Geral de Filmes em Treino:', filmes_treino)
print('Total Geral de Filmes que NÃO estão em treino: {} ({}%)'.format(novos_filmes,
                                                                         np.round((novos_filmes / total_movies)*100, 2)))

346 filmes não aparecem nos dados de treino. Teremos que lidar com isso quando trabalharmos no modelo de ML

*CALCULANDO A MATRIZ DE SIMLIARIDADE DE USUÁRIOS*

In [ ]:
#Função de calculo de similaridade
def calc_similaridade_user(sparse_matrix,
                           compute_for_few = False,
                           top = 100,
                           verbose = False,
                           verb_for_n_rows = 20,
                           draw_time_taken = True):
    #Variaveis de controle
    no_of_users, _ = sparse_matrix.shape
    row_ind, col_ind = sparse_matrix.nonzero()
    row_ind = sorted(set(row_ind))
    time_taken = []
    rows, cols, data = [], [], []
    if verbose: print("Calculando top", top, "similaridades para cada usuário...")
    start = datetime.now()
    temp = 0
    
    #Loop pela matriz
    for row in row_ind[:top] if compute_for_few else row_ind:
        temp = temp +1
        prev = datetime.now()
        #Calculando similaridade de cosseno
        sim = cosine_similarity(sparse_matrix.getrow(row), sparse_matrix).ravel()
        top_sim_ind = sim.argsort()[-top:]
        top_sim_val = sim[top_sim_ind]
        rows.extend([row]*top)
        cols.extend(top_sim_ind)
        data.extend(top_sim_val)
        time_taken.append(datetime.now().timestamp()-prev.timestamp())
        
        if verbose:
            if temp%verb_for_n_rows == 0:
                print("Cálculo concluído para {} usuários [ tempo total: {} ]".format(temp, datetime.now()-start))
                
    if verbose: print('Criação de matriz esparsa a partir das semelhanças computadas...')
    
    if draw_time_taken:
        plt.plot(time_taken, label = 'Tempo de cálculo de cada usuário')
        plt.plot(np.cumsum(time_taken), label = 'Tempo Total')
        plt.legend(loc = 'best')
        plt.xlabel('Usuário')
        plt.ylabel('Tempo (segundos)')
        plt.show()
    return sparse.csr_matrix((data, (rows, cols)), shape = (no_of_users, no_of_users)), time_taken            

In [ ]:
#Calculando a similaridade
#Marca o início
start = datetime.now()

#Calcula a similaridade
matriz_esparsa_user, _ = calc_similaridade_user(matriz_esparsa_treino,
                                                compute_for_few= True,
                                                top = 100,
                                                verbose = True)

print("Tempo Total de Processamento:", datetime.now()-start)

Tentaremos reduzir a dimensionalidade usando SVD

*REDUÇÃO DE DIMENSIONALIDADE COM TRUNCATEDSVD*

In [ ]:
#Redução de dimensionalidae
#Marca o ínicio
start = datetime.now()

#Cria o objeto TrucatedSVD reduzindo a dimensionalidade para 500 dims.
netflix_svd = TruncatedSVD(n_components= 500, algorithm='randomized', random_state= 15)

#Aplica o TruncatedSVD
trunc_svd = netflix_svd.fit_transform(matriz_esparsa_treino)

print("Tempo total de processamento:", datetime.now() - start)

*CALCULAR A VARIANCIA EXPLICADA PELOS COMPONENTES*

In [ ]:
#Calcula a variancia explicada
expl_var = np.cumsum(netflix_svd.explained_variance_ratio_)


In [ ]:
#Plot
fig, (ax1) = plt.subplots(nrows= 1, ncols = 1, figsize = plt.figaspect(.45))

ax1.set_ylabel("Variância Explicada", fontsize = 15)
ax1.set_xlabel("Fatores Latentes", fontsize = 15)
ax1.plot(expl_var)

#Marcar algumas combinações para ficar mais claro
ind = [1, 2, 4, 8, 20, 60, 100, 200, 300, 400, 500]
ax1.scatter(x = [i-1 for i in ind], y = expl_var[[i-1 for i in ind]],c = '#ee4422')

for i in ind:
    ax1.annotate(s = "({}, {}".format(i, np.round(expl_var[i-1], 2)), xy = (i-1, expl_var[i-1]),
                 xytext = (i+20, expl_var[i-1] - 0.01), fontweight = 'bold')
    
plt.show()

In [ ]:
#Projetando a matriz no espaço de 500 dims.
trunc_matrix = matriz_esparsa_treino.dot(netflix_svd.components_.T)

In [ ]:
#Shape
trunc_matrix.shape

In [ ]:
#TIpo
type.(trunc_matrix)

In [ ]:
#Salvando em disco a matriz com a dimensionalidade reduzida pra 500
if not os.path.isfile('C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/matriz_esparsa_user_truncada.npz'):
    matriz_esparsa_user_truncada = sparse.csr_matrix(trunc_matrix)
    sparse.save_npz('C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/matriz_esparsa_user_truncada.npz', matriz_esparsa_user_truncada)
else:
    matriz_esparsa_user_truncada = sparse.load_npz('C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/matriz_esparsa_user_truncada.npz')

In [ ]:
#shape
matriz_esparsa_user_truncada.shape

*Calcular a similaridade com a matriz truncada*

In [ ]:
#Calcula a similaridade de usuários
trunc_sim_matrix, _ = calc_similaridade_user(matriz_esparsa_user_truncada,
                                             compute_for_few=True,
                                             top = 50,
                                             verbose = True)

*CALCULANDO A MATRIZ DE SIMILARIDADE PARA FILMES*

In [ ]:
#Cria se não existir
if not os.path.isfile('C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/matriz_esparsa_filme.npz'):
    matriz_esparsa_filme = cosine_similarity(x = matriz_esparsa_treino.T, dense_output = False)
    sparse.save_npz('C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/matriz_esparsa_filme.npz')
else:
    matriz_esparsa_filme = sparse.load_npz('C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/matriz_esparsa_filme.npz')



In [ ]:
#shape
matriz_esparsa_filme.shape

In [ ]:
#extrai os ids dos filmes
movie_ids = np.unique(matriz_esparsa_filme.nonzero()[1])

In [ ]:
#Calcula a similaridade de filmes de acordo com o padrão de avaliação do usuário
filmes_similares = {}

#loop pelos ids dos filmes
for movie in movie_ids:
    #obtemos os top filmes semelhantes e armazenamos no dicionário
    filmes_sim = matriz_esparsa_filme[movie].toarray().ravel().argsort()[::-1][1:]
    filmes_similares[movie] = filmes_sim[:100]

In [ ]:
#Filmes similares ao filme de id 43
filmes_similares[43]

*ENCONTRAR OS FILMES SEMELHANTES USANDO A MATRIZ DE SIMILARIDADE

In [ ]:
#Vamos carregar os títulos dos filmes do arquivo csv fornecido pela Netflix
titulos_filmes = pd.read_csv('C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/movies_titles.csv',
                             sep = ',',
                             header = None,
                             names = ['ID_Filme', 'Ano_Lancamento','Titulo'],
                             verbose = True,
                             index_col = 'ID_Filme',
                             encoding = 'ISO-8859-1')

In [ ]:
#Visualiza dados
titulos_filmes.head()

*Filmes similares ao filme 43*

In [ ]:
#ID do filme
id_filme = 43
print("Filme", titulos_filmes.loc[id_filme].values[1])
print("Total de avaliações de usuários = {}.".format(matriz_esparsa_treino[:,id_filme].getnnz()))
print("Encotramos {} filmes que são similares a este e vamos imprimir os mais similares".format(matriz_esparsa_filme[:,id_filme].getnnz()))

In [ ]:
#Encontrando todas similaridads
similarities = matriz_esparsa_filme[id_filme].toarray().ravel()
similar_indices = similarities.argsort()[::-1][1:]
similarities[similar_indices]
sim_indices = similarities.argsort()[::-1][1:]


In [ ]:
#plot
fig = plt.figure(figsize = plt.figaspect(.45))
plt.plot(similarities[sim_indices], label = 'Todas as Avaliações')
plt.plot(similarities[sim_indices[:100]], label = 'Top 100 Filmes Similares')
plt.title("Filmes Similares ao Filme {}".format(id_filme), fontsize=25)
plt.xlabel("Filmes", fontsize = 15)
plt.ylabel("Similaridade de Cosseno", fontsize = 15)
plt.legend()
plt.show()

In [ ]:
#Top 10 filmes similares ao filme 43
titulos_filmes.loc[sim_indices[:10]]

*Até aqui fizemos o nosso sistema de recomendação. Agora iniciaremos a construção de um modelo de ML a fim de prever a avaliação que o usuário atribuirá a um filme.*

In [ ]:
#Importando os pacotes 
import os
import random
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import scipy
from scipy import sparse
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
import xgboost as xgb
from datetime import datetime

Vamos construir uma função para extrair amostras dos dados, já que, se trata de muitos dados e demoraria muito tempo para executar tudo de uma vez

In [ ]:
#Função para extrair amostra da matriz esparsa
from random import sample


def gera_amostra(sparse_matrix, num_users, num_movies, path, verbose = True):
    #Tupla: row, col e rating da matriz esparsa
    row_ind, col_ind, ratings = sparse.find(sparse_matrix)
    users = np.unique(row_ind)
    movies = np.unique(col_ind)
    
    #random seed para reproduzir o processo aleatorio
    np.random.seed(15)
    
    #Amostras de usuarios e filmes
    sample_users = np.random.choice(users, num_users, replace = True)
    sample_movies = np.random.choice(movies, num_movies, replace = True)
    
    #Gera mascara boolena
    mask = np.logical_and(np.isin(row_ind, sample_users), np.isin(col_ind, sample_movies))
    
    #matriz esparsa com as amostras da matriz original
    amostra_matriz_esparsa = sparse.csr_matrix((ratings[mask], (row_ind[mask], col_ind[mask])),
                                               shape = (max(sample_users)+1, max(sample_movies)+1))
    
    #Salva em disco
    sparse.save_npz(path, amostra_matriz_esparsa)
    
    if verbose:
        print('Tarfe concluída. \n')
        
        
    return amostra_matriz_esparsa

*Gerando a amostra de treino*

In [ ]:
#Gerando uma amostra

#caminho onde esta a matriz original
caminho_original_treino =  'C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/matriz_esparsa_treino.npz'

#carregando a matriz
matriz_loaded_treino = sparse.load_npz(caminho_original_treino)

#onde salvar a amostra
path = 'C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/amostra_matriz_esparsa_treino.npz'

#obtemos uma amostra de 10000 usuários e 1000 filmes
amostra_treino = gera_amostra(matriz_loaded_treino, 
                              num_users=10000, 
                              num_movies=1000, 
                              path = path)


Gerando amostra de teste

In [ ]:
#Gerando uma amostra

#caminho onde está a matriz original
caminho_original_teste =  'C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/matriz_esparsa_teste.npz'

#carregando a matriz
matriz_loaded_teste = sparse.load_npz(caminho_original_teste)

#onde salvar a amostra
path = 'C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/amostra_matriz_esparsa_teste.npz'

#obtemos uma amostra de 2000 usuários e 200 filmes
amostra_teste = gera_amostra(matriz_loaded_teste, 
                              num_users=2000, 
                              num_movies=200, 
                              path = path)

In [ ]:
#resumo
print('Numero de amostras na matriz de amostras de TREINO: {}'.format(amostra_treino.count_nonzero()))
print('Numero de amostras na matriz de amostras de TESTE: {}'.format(amostra_teste.count_nonzero()))

*Métricas*

In [ ]:
#cria um dicionário
amostra_medias_treino = {}

Função para calcular a media de avaliações

In [ ]:
from audioop import avg


def calcula_media_ratings(sparse_matrix, of_users):
    #media de avaliações
    #1 eixo de users
    #0 eixo de filmes
    ax = 1 if of_users else 0
    
    #soma das avaliaçoes
    sum_of_ratings = sparse_matrix.sum(axis=ax).A1
    
    #Matriz boolena de avaliações(se o usuario avaliou aquele filme ou nao)
    is_rated = sparse_matrix != 0
    
    #num de avaliaçoes de cada usuario ou filme
    no_of_ratings = is_rated.sum(axis = ax).A1
    
    #Ids da matriz esparsa, u de user m de movie
    u, m = sparse_matrix.shape
    
    #dict de users e suas avaliaçoes
    avg_ratings = {i:sum_of_ratings[i] / no_of_ratings[i]
                   for i in range(u if of_users else m)
                   if no_of_ratings[i] != 0}
    
    return avg_ratings

Media global das avaliaçoes

In [ ]:
#media global
mediaglobal = amostra_treino.sum() / amostra_treino.count_nonzero()
amostra_treino['global'] = mediaglobal
amostra_medias_treino

Media por usuario

In [ ]:
#media de avaliaçao dos usuarios
amostra_medias_treino['user'] = calcula_media_ratings(amostra_treino, of_users= True)

In [ ]:
#extrair um dos usuarios do dict de filmes, o objetivo é so automatizar o processo
um_usuario = [a for a, b in amostra_medias_treino['user'].items()][0]
um_usuario

In [ ]:
print('Media de Avaliação do Usuario ' + str(um_usuario) + ':', amostra_medias_treino['user'][um_usuario])

Media por filme

In [ ]:
#media de avaliaçao dos usuarios
amostra_medias_treino['movie'] = calcula_media_ratings(amostra_treino, of_users= False)

In [ ]:
#extrair um dos usuarios do dict de filmes, o objetivo é so automatizar o processo
um_filme = [a for a, b in amostra_medias_treino['movie'].items()][0]
um_filme

In [ ]:
print('Media de Avaliação do Filme ' + str(um_filme) + ':', amostra_medias_treino['movie'][um_filme])

*Formatando os dados*

-variaveis de entrada-
GAvg -> media global das avals
sur1, sur2... -> (avaliação de usuarios semelhantes)
smr1, smr2... -> (filmes semelhantes avaliados por um usuario)
UAvg -> media das avaliaçoes dos usuarios
MAvg -> media das avaliaçoes do filme

-Variavel Target-
rating -> avaliação do filme dada pelo usuario

*PREPARANDO OS DADOS DE TREINO PARA O MODELO DE REGRESSÃO*

In [ ]:
#extraindo os dados da matriz de amostras
amostra_usuarios_treino, amostras_filmes_treino, amostra_avaliacoes_treino = sparse.find(amostra_treino)

a celula abaixo demora muito para ser executada

In [ ]:
#verificando se o arquivo ja existe

if os.path.isfile('C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/dados_treino_reg.csv'):
    print('O arquivo ja existe e nao precisa ser criado novamente')
else:
    print('Preparando {} tuplas para o dataset... \n'.format(len(amostra_medias_treino)))
    with open('C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/dados_treino_reg.csv', mode = 'w') as reg_data_file:
        count = 0
        for (user, movie, rating) in zip(amostra_usuarios_treino,amostras_filmes_treino, amostra_avaliacoes_treino):
            #### avaliaçao de um filme por usuarios similares ao usuario corrente ####
            #calcula usuario similar ao usuario corrente
            user_sim = cosine_similarity(amostra_treino[user],amostra_treino).ravel()
            
            #obtem top user
            top_sim_user = user_sim.argsort()[::-1][1:]
            
            #top ratings
            top_ratings = amostra_treino[top_sim_user, movie].toarray().ravel()
            
            #top 5 usuarios similares
            top_sim_user_ratings = list(top_ratings[top_ratings != 0][:5])
            top_sim_user_ratings.extend([amostra_treino['movie'][movie]]*(5 - len(top_sim_user_ratings)))
            
            
            #### avaliaçao por usuario para filmes similares ao filme corrente ####
            #calcula filme similar ao filme corrente
            movie_sim = cosine_similarity(amostra_treino[movie],amostra_treino).ravel()
            
            #obtem top movie
            top_sim_movie = movie_sim.argsort()[::-1][1:]
            
            #top ratings
            top_ratings = amostra_treino[top_sim_movie, movie].toarray().ravel()
            
            #top 5 usuarios similares
            top_sim_movie_ratings = list(top_ratings[top_ratings != 0][:5])
            top_sim_movie_ratings.extend([amostra_treino['movie'][movie]]*(5 - len(top_sim_movie_ratings)))
            
            #### prepara a linha que sera armazenada no arquivo ####
            row = []
            row.append(user)
            row.append(movie)
            
            #Adicionamos outros atributos
            row.append(amostra_medias_treino['global'])
            row.extend(top_sim_user_ratings)
            row.extend(top_sim_movie_ratings)
            row.append(amostra_medias_treino['user'][user])
            row.append(amostra_medias_treino['movie'][movie])
            
            row.append(rating)
            count = count + 1
            
            #if count == 10: 
                #break 
            
            reg_data_file.write(','.join(map(str,row)))
            reg_data_file.write('\n')
            if (count)%10000 == 0:
                print("Concluido para {} linhas----- {}".format(count, datetime.now() - start))

In [ ]:
#Carregamos o arquivo e colocamos em um DF
from os import getloadavg


df_dados_treino_reg = pd.read_csv('C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/dados_treino_reg.csv',
                                  names = ['user',
                                  'movie',
                                  'GAvg', 
                                  'sur1',
                                  'sur2',
                                  'sur3',
                                  'sur4',
                                  'sur5',
                                  'smr1',
                                  'smr2',
                                  'smr3',
                                  'smr4',
                                  'smr5',
                                  'UAvg',
                                  'MAvg',
                                  'rating'], header = None)

In [ ]:
#dados
df_dados_treino_reg.head()

*PREPARANDO OS DADOS DE TESTE PARA O MODELO*

In [ ]:
#extraindo os dados da matriz de amostras
amostra_usuarios_teste, amostras_filmes_teste, amostra_avaliacoes_teste = sparse.find(amostra_teste)

In [ ]:
if os.path.isfile('C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/dados_teste_reg.csv'):
    print("O arquivo já existe e não precisa ser criando novamente")
else:
    print("Preparando {} tuplas para o dataset..\n".format(len(amostra_teste)))
    with open('C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/dados_teste_reg.csv', mode = 'w') as reg_data_file:
        count = 0
        for (user, movie, rating) in zip(amostra_usuarios_teste, amostras_filmes_teste, amostra_avaliacoes_teste):
            st = datetime.now()
            
            #similaridade de usuarios
            try:
                user_sim = cosine_similarity(amostra_treino[user], amostra_treino).ravel()
                top_sim_user = user_sim.argsort()[::-1][1:]
                top_ratings = amostra_treino[top_sim_user, movie].toarray().ravel()
                top_sim_user_ratings = list(top_ratings[top_ratings !=0 ][:5])
                top_sim_user_ratings.extend([amostra_medias_treino['movie'][movie]]*(5 - len(top_sim_user_ratings)))
                
            except (IndexError, KeyError):
                top_sim_user_ratings.extend([amostra_medias_treino['global']]*(5 - len(top_sim_user_ratings)))
            except:
                print(user,movie)
                raise
            
            #Similaridade dos filmes
            try:
                movie_sim = cosine_similarity(amostra_treino[:,movie].T,
                                              amostra_treino.T).ravel()
                top_sim_movie = movie_sim.argsort()[::-1][1:]
                top_ratings = amostra_treino[user, top_sim_movie].toarray().ravel()
                top_sim_movie_ratings = list(top_ratings[top_ratings !=0 ][:5])
                top_sim_movie_ratings.extend([amostra_medias_treino['user'][user]]*(5 - len(top_sim_movie_ratings)))
            except(IndexError, KeyError):
                top_sim_movie_ratings.extend([amostra_medias_treino['global']]*(5 - len(top_sim_movie_ratings)))
            except:
                raise
            
            #prepara os dados para gravar o arquivo
             # Prepara os dados para gravar no arquivo
            row = list()
            row.append(user)
            row.append(movie)
            row.append(amostra_medias_treino['global']) 
            row.extend(top_sim_user_ratings)
            row.extend(top_sim_movie_ratings)
            
            try:
                row.append(amostra_medias_treino['user'][user])
            except KeyError:
                row.append(amostra_medias_treino['global'])
            except:
                raise
            
            row.append(rating)
            
            count = count +1
            
            #if count == 5
                #break
            
            reg_data_file.write(','.join(map(str,row)))
            reg_data_file.write('\n')
            if(count)%1000 == 0
            print('Concluido em {} linhas---- {}'.format(count, datetime.now() - start))

In [ ]:
# Gera o dataset de teste
df_dados_teste_reg = pd.read_csv('dados/dados_teste_reg.csv', names = ['user', 
                                                                       'movie', 
                                                                       'GAvg', 
                                                                       'sur1', 
                                                                       'sur2', 
                                                                       'sur3', 
                                                                       'sur4', 
                                                                       'sur5',
                                                                       'smr1', 
                                                                       'smr2', 
                                                                       'smr3', 
                                                                       'smr4', 
                                                                       'smr5',
                                                                       'UAvg', 
                                                                       'MAvg', 
                                                                       'rating'], 
                                 header = None)

In [ ]:
df_dados_teste_reg.head()

*CONSTRUINDO O MODELO DE MACHINE LEARNING*

In [ ]:
#dict para avaliaçao do modelo
models_evaluation_train = {}
models_evaluation_test = {}

In [ ]:
#funçao para calculo de erro do modelo
def calcula_metricas(y_true, y_pred):
    rmse = np.sqrt(np.mean([(y_true[i] - y_pred[i])**2 for i in range(len(y_pred)) ] ))
    mape = np.mean(np.abs( (y_true - y_pred)/ y_true)) * 100
    return rmse, mape

In [ ]:
#funçao para treino do modelo
def exec_xgb(model, x_train, y_train, x_test, y_test, verbose = True):
    
    #dicts
    train_results = {}
    test_results = {}
    
    #treino 
    print("Treinando o modelo")
    model.fit(x_train, y_train, eval_metric = 'rmse')
    print("Concluído")
    
    #calculando o erro do modelo nos dados de treino
    print('Calculando metricas em Treino')
    y_train_pred = model.predict(x_train)
    rmse_train, mape_train = calcula_metricas(y_train.values, y_train_pred)
    
    #grava os resultados
    train_results = {'rmse' : rmse_train, 'mape': mape_train, 'previsoes' : y_train_pred}
    
    if verbose:
        print('\nErro do modelo em treino')
        print('-'*30)
        print('RMSE: ', rmse_train)
        print('MAPE: ' mape_train)
        
    #avaliando o modelo em dados de teste
    print('\nAvaliando o modelo em dados de teste')
    y_test_pred = model.predict(x_test)
    rmse_test, mape_test = calcula_metricas(y_true = y_test.values, y_pred = y_test_pred)
    
    #grava resultados
    test_results = {'rmse' : rmse_test, 'mape': mape_test, 'previsoes':y_test_pred}
    if verbose:
        print('\nErro do modelo em teste')
        print('-'*30)
        print('RMSE: ', rmse_test)
        print('MAPE: ' mape_test)
    
    return train_results, test_results

In [ ]:
#seed
my_seed = 15
random.seed(my_seed)
np.random.seed(my_seed)

*TREINAMENTO DO MODELO*

In [ ]:
#preparando dados de treino
x_treino = df_dados_treino_reg.drop(['user','movie','rating'], axis = 1)
y_treino = df_dados_treino_reg['rating']

In [ ]:
# Prepara os dados de teste
x_teste = df_dados_teste_reg.drop(['user', 'movie', 'rating'], axis = 1)
y_teste = df_dados_teste_reg['rating']

In [ ]:
#Cria o modelo xgboost regressor com 100 estimadores
model_xgb = xgb.XGBRegressor(silent = False, random_state = 15, n_estimators = 100)

In [ ]:
#treinamento do modelo
train_results, test_results = exec_xgb(model_xgb, x_treino, y_treino, x_teste, y_teste)

In [ ]:
# Armazena os resultados da avaliação do modelo
models_evaluation_train['modelo_xgb'] = train_results
models_evaluation_test['modelo_xgb'] = test_results

In [ ]:
#variaveis mais importantes
xgb.plot_importance(model_xgb)
plt.show()

Além de construir o modelo também identificamos as variáveis mais relevantes. Observe que não há surpresa. As avaliações de usuários são determinantes para recomendar os filmes avaliados para outros usuários.

*SALVANDO O RESULTADO*

In [ ]:
#Salva os resultados em disco
pd.DataFrame(models_evaluation_test).to_csv('C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/resultado.csv')
models = pd.read_csv('C:/Users/est.diegosv/OneDrive - Votorantim/Diego/ML/SistemaRecomendacao_Netflix/resultado.csv', index_col = 0)
models.loc['rmse'].sort_values()